# Segunda entrega - Proyecto de Analítica

**Integrantes:**

- Jhon Alexander Bedoya Carvajal
- Anibal Sebastián Salamanca Mendez
- Brahayan Stiven Gil Henao 

# 1. Carga de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import json

# 2. Carga de datos

## 2.1. Cargar base de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_vh = pd.read_csv('/content/drive/MyDrive/Introducción a la Analítica/Proyecto/Entrega 1/vehicle.csv')
df_acc = pd.read_csv('/content/drive/MyDrive/Introducción a la Analítica/Proyecto/Entrega 1/accident.csv')
df_pr = pd.read_csv('/content/drive/MyDrive/Introducción a la Analítica/Proyecto/Entrega 1/person.csv')
vh_mk = pd.read_csv('/content/drive/MyDrive/Introducción a la Analítica/Proyecto/Entrega 1/Vehicle Make.csv', sep=';')

<ipython-input-419-5f30dc48397e>:1: DtypeWarning:

Columns (101,102,103,106,107,108,109,116,117,119,124,125,128) have mixed types. Specify dtype option on import or set low_memory=False.

<ipython-input-419-5f30dc48397e>:3: DtypeWarning:

Columns (71,72,73,74,81,82,84,88,89,93) have mixed types. Specify dtype option on import or set low_memory=False.



## 2.2. Preguntas

1. ¿Cuál es la cantidad de accidentes de tránsito con muertos por condado?
2. Muestre la cantidad de accidentes de tránsito por año-mes y la cantidad de estos que registran muertos.
3. ¿Cuántos accidentes de tránsito se registran por hora del día, en promedio? ¿Cuántos de estos registran muertos?
4. Indique si hay alguna relación entre los accidentes de tránsito y el estado del clima.
5. ¿Los vehículos más antiguos son los que causan más accidentes? ¿O es el caso contrario?
6. ¿Qué causa más muertes en accidentes de tránsito? ¿Conductores bajo efectos de alcohol o bajo efectos de drogas?
7. ¿Cuántas personas de cada sexo mueren en accidentes de tránsito por año? ¿Cuál es el promedio de edad?
8. ¿Total de accidentes y muertes por marca de vehiculo?
9. ¿La activación del Airbag disminuye la cantidad de muertes en promedio? ¿Que tipo de activación es  mejor? 

## 2.3. Seleccionar variables de interes

In [ ]:
acc = df_acc[['accident_id', 'county_name', 'DAY', 'MONTH', 'YEAR', 'HOUR', 'MINUTE',
       'LATITUDE', 'LONGITUD',  'sp_jur_lit',  'man_coll_lit', 'lgt_cond_lit',  'weather_lit', 'FATALS']]

vh = df_vh[['accident_id', 'MOD_YEAR', 'VEH_NO', 'MAKE']]

pr = df_pr[['accident_id', 'AGE', 'SEX', 'INJ_SEV', 'DRINKING', 'ALC_RES', 'DRUGS', 'DRUGRES1', 'DRUGRES2', 'DRUGRES3', 'AIR_BAG', 'VEH_NO']]

# 3. Depuración

In [ ]:
acc.columns = acc.columns.str.lower()
vh.columns = vh.columns.str.lower()
pr.columns = pr.columns.str.lower()

## 3.1. Homologación de categorías

### 3.1.1. Tabla accidentes

In [ ]:
acc.shape

(3904, 14)

In [ ]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3904 entries, 0 to 3903
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   accident_id   3904 non-null   int64  
 1   county_name   3904 non-null   object 
 2   day           3904 non-null   int64  
 3   month         3904 non-null   int64  
 4   year          3904 non-null   int64  
 5   hour          3904 non-null   int64  
 6   minute        3904 non-null   int64  
 7   latitude      3904 non-null   float64
 8   longitud      3904 non-null   float64
 9   sp_jur_lit    3904 non-null   object 
 10  man_coll_lit  3904 non-null   object 
 11  lgt_cond_lit  3904 non-null   object 
 12  weather_lit   3904 non-null   object 
 13  fatals        3904 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 427.1+ KB


**city_name**

In [ ]:
acc['county_name'].unique()

array(['PIMA', 'YAVAPAI', 'MARICOPA', 'NAVAJO', 'SANTA CRUZ', 'GRAHAM',
       'PINAL', 'COCONINO', 'MOHAVE', 'GILA', 'YUMA', 'APACHE', 'COCHISE',
       'LA PAZ', 'GREENLEE', '999'], dtype=object)

Se observa una categoría extraña: 999. Este valor se utiliza para indicar que no hay registro del condado donde ocurrió el accidente.

In [ ]:
acc[acc['county_name']=='999']

,accident_id,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals
733,2012040750,999,31,10,2012,99,99,99.9999,999.9999,Unknown,Not a Collision with Motor Vehicle In-Transport,Unknown,Unknown,1
1069,2013040342,999,7,5,2013,99,99,99.9999,999.9999,Indian Reservation,Unknown,Unknown,Unknown,1
1491,2013040784,999,13,12,2013,99,99,99.9999,999.9999,Unknown,Not a Collision with Motor Vehicle In-Transport,Unknown,Unknown,1
2949,2015040749,999,7,11,2015,99,99,99.9999,999.9999,Unknown,Unknown,Unknown,Unknown,1
3026,2015040830,999,25,7,2015,99,99,99.9999,999.9999,Unknown,Not a Collision with Motor Vehicle In-Transport,Unknown,Unknown,1
3281,2016040248,999,1,4,2016,12,30,99.9999,999.9999,Unknown,Not a Collision with Motor Vehicle In-Transport,Unknown,Not Reported,1


Los valores 999 es el valor que utilizan para los valores nulos en esta columna, así lo trataremos.

In [ ]:
acc.loc[:, ['county_name']] = acc.loc[:,['county_name']].replace({'999':np.nan})

<ipython-input-739-165ecd854263>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
acc['county_name'].unique()

array(['PIMA', 'YAVAPAI', 'MARICOPA', 'NAVAJO', 'SANTA CRUZ', 'GRAHAM',
       'PINAL', 'COCONINO', 'MOHAVE', 'GILA', 'YUMA', 'APACHE', 'COCHISE',
       'LA PAZ', 'GREENLEE', nan], dtype=object)

**man_coll_lit:** Manera de colisión.

In [ ]:
acc['man_coll_lit'].unique()

array(['Not a Collision with Motor Vehicle In-Transport', 'Angle',
       'Front-to-Rear', 'Sideswipe - Opposite Direction', 'Unknown',
       'Sideswipe - Same Direction', 'Front-to-Front', 'Rear-to-Side',
       'Other'], dtype=object)

La categoría 'Unknown' se agregará a la categoría 'Other'.

In [ ]:
acc.groupby(['man_coll_lit'])[['accident_id']].count()

,accident_id
man_coll_lit,
Angle,632
Front-to-Front,247
Front-to-Rear,254
Not a Collision with Motor Vehicle In-Transport,2516
Other,8
Rear-to-Side,9
Sideswipe - Opposite Direction,26
Sideswipe - Same Direction,69
Unknown,143


In [ ]:
acc['man_coll_lit'] = acc['man_coll_lit'].replace({'Unknown' : 'Other'})

<ipython-input-743-e27b90acec29>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
acc['man_coll_lit'].unique()

array(['Not a Collision with Motor Vehicle In-Transport', 'Angle',
       'Front-to-Rear', 'Sideswipe - Opposite Direction', 'Other',
       'Sideswipe - Same Direction', 'Front-to-Front', 'Rear-to-Side'],
      dtype=object)

**lgt_cond_lit:** Condición de luz al momento del accidente.

In [ ]:
acc['lgt_cond_lit'].unique()

array(['Dark - Not Lighted', 'Daylight', 'Dark - Lighted', 'Dusk', 'Dawn',
       'Dark - Unknown Lighting', 'Unknown', 'Not Reported'], dtype=object)

La categoría 'Unknown' se agregará a la categoría 'Not reported'.

In [ ]:
acc.groupby(['lgt_cond_lit'])[['accident_id']].count()

,accident_id
lgt_cond_lit,
Dark - Lighted,869
Dark - Not Lighted,702
Dark - Unknown Lighting,195
Dawn,107
Daylight,1730
Dusk,122
Not Reported,8
Unknown,171


In [ ]:
 acc.loc[:,['lgt_cond_lit']] = acc.loc[:,['lgt_cond_lit']].replace({'Unknown' : 'Not Reported'})

<ipython-input-747-077856aa2587>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
acc['lgt_cond_lit'].unique()

array(['Dark - Not Lighted', 'Daylight', 'Dark - Lighted', 'Dusk', 'Dawn',
       'Dark - Unknown Lighting', 'Not Reported'], dtype=object)

**weather_lit:** Condición del clima al momento de accidente.

In [ ]:
acc['weather_lit'].unique()

array(['Clear', 'Cloudy', 'Rain', 'Severe Crosswinds', 'Unknown',
       'Blowing Sand, Soil, Dirt', 'Not Reported', 'Snow', 'Blowing Snow',
       'Sleet or Hail', 'Other', 'Fog, Smog, Smoke'], dtype=object)

In [ ]:
acc.groupby(['weather_lit'])[['accident_id']].count()

,accident_id
weather_lit,
"Blowing Sand, Soil, Dirt",5
Blowing Snow,1
Clear,2953
Cloudy,360
"Fog, Smog, Smoke",6
Not Reported,189
Other,1
Rain,88
Severe Crosswinds,6


Las categorías 'Unknown' y 'Other' se agregarán a la categoría 'Not Reported'.

La categoría 'Blowing Snow' se agregará a la categoría 'Snow'.

La categoría Fog, Smog, Smoke' (Niebla, Smog, Humo) se agregará a la categoría 'Cloudy' (Nublado).

Se renombra la categoría 'Blowing Sand, Soil, Dirt' a solo 'Blowing Sand'.

In [ ]:
acc.loc[:,['weather_lit']] = acc.loc[:,['weather_lit']].replace({'Unknown' : 'Not Reported',
                                                                 'Other':'Not Reported', 
                                                                 'Blowing Snow':'Snow', 
                                                                 'Fog, Smog, Smoke':'Cloudy',
                                                                 'Blowing Sand, Soil, Dirt': 'Blowing Sand'})

<ipython-input-751-4c20a1e00aed>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
acc.groupby(['weather_lit'])[['accident_id']].count()

,accident_id
weather_lit,
Blowing Sand,5
Clear,2953
Cloudy,366
Not Reported,475
Rain,88
Severe Crosswinds,6
Sleet or Hail,4
Snow,7


**year, month, day, hour y minute**

In [ ]:
for col in ['year', 'month', 'day', 'hour', 'minute']:
    print(col, np.sort(acc[col].unique()), '\n')

year [2012 2013 2014 2015 2016] 

month [ 1  2  3  4  5  6  7  8  9 10 11 12] 

day [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31] 

hour [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 99] 

minute [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 99] 



Los valores de 99 en las columnas "hour" y "minute" indican nulos.

In [ ]:
acc.loc[:, 'hour'] = acc.loc[:,'hour'].replace({99 : np.nan})
acc.loc[:,'minute'] = acc.loc[:,'minute'].replace({99 : np.nan})

<ipython-input-754-5422d45804ab>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-754-5422d45804ab>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**latitude y longitud**

In [ ]:
acc[(acc['latitude']==99.9999) | (acc['longitud']==999.9999)]

,accident_id,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals
52,2012040054,PIMA,22,1,2012,NaN,NaN,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
102,2012040105,COCONINO,12,2,2012,NaN,NaN,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
163,2012040170,APACHE,13,3,2012,1.0,34.0,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Dark - Unknown Lighting,Not Reported,1
164,2012040171,NAVAJO,13,3,2012,16.0,33.0,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Not Reported,1
176,2012040184,MARICOPA,25,3,2012,17.0,46.0,99.9999,999.9999,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
238,2012040247,COCONINO,20,4,2012,18.0,13.0,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,2
242,2012040251,APACHE,24,4,2012,20.0,30.0,99.9999,999.9999,Indian Reservation,Front-to-Front,Dark - Not Lighted,Clear,1
252,2012040262,PIMA,6,5,2012,11.0,45.0,99.9999,999.9999,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1
292,2012040303,NAVAJO,5,5,2012,15.0,8.0,99.9999,999.9999,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1
306,2012040318,APACHE,27,5,2012,11.0,0.0,99.9999,999.9999,National Park Service,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1


Loa valores de 99.9999 en la columna "latitude" y los valores de 999.9999 en la columna "longitud" indican nulos.

In [ ]:
acc['latitude'] = acc['latitude'].replace({99.9999 : np.nan})
acc['longitud'] = acc['longitud'].replace({999.9999 : np.nan})

<ipython-input-756-174df948c14a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-756-174df948c14a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 3.2.2. Tabla vehículos

In [ ]:
vh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5889 entries, 0 to 5888
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   accident_id  5889 non-null   int64
 1   mod_year     5889 non-null   int64
 2   veh_no       5889 non-null   int64
 3   make         5889 non-null   int64
dtypes: int64(4)
memory usage: 184.2 KB


**mod_year:** Año de modelo del vehículo.

In [ ]:
np.sort(vh['mod_year'].unique())

array([1932, 1956, 1957, 1963, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 9999])

Los valores 9999 se reemplazarán por nulos para tratarlos más adelante.

In [ ]:
vh['mod_year'] = vh['mod_year'].replace({9999: np.nan})

<ipython-input-759-a8abb226eb97>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
vh.groupby(['make'])[['accident_id']].count()

Se dejará las categorías "Not reported" y "Unknown make" como "Unknown".

In [ ]:
vh['make'] = vh['make'].replace([97, 99], 'Unknown')

<ipython-input-837-b875c79d79be>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### 3.2.3. Tabla personas

In [ ]:
pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10611 entries, 0 to 10610
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   accident_id  10611 non-null  int64
 1   age          10611 non-null  int64
 2   sex          10611 non-null  int64
 3   inj_sev      10611 non-null  int64
 4   drinking     10611 non-null  int64
 5   alc_res      10611 non-null  int64
 6   drugs        10611 non-null  int64
 7   drugres1     10611 non-null  int64
 8   drugres2     10611 non-null  int64
 9   drugres3     10611 non-null  int64
 10  air_bag      10611 non-null  int64
 11  veh_no       10611 non-null  int64
dtypes: int64(12)
memory usage: 994.9 KB


- **sex:** Sexo de la persona.

            | Codes | Attributes     |
            |-------|----------------|
            | 1     | Male           |
            | 2     | Female         |
            | 8     | Not Reported   |
            | 9     | Unknown        |

In [ ]:
pr['sex'].unique()

array([1, 2, 9, 8])

In [ ]:
pr.loc[:,['sex']] = pr.loc[:,['sex']].replace( {1:'Male', 2:'Female', 8:'Not Reported', 9 :'Not Reported'})

<ipython-input-762-772e9a326bce>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.groupby(['sex'])[['accident_id']].count()

,accident_id
sex,
Female,3632
Male,6674
Not Reported,305


- **inj_sev:** Severidad de las heridas:

            | Codes | Attributes                     |
            |-------|--------------------------------|
            | 0     | No Apparent Injury (O)         |
            | 1     | Possible Injury (C)            |
            | 2     | Suspected Minor Injury (B)     |
            | 3     | Suspected Serious Injury (A)   |
            | 4     | Fatal Injury (K)               |
            | 5     | Injured, Severity Unknown      |
            | 6     | Died Prior to Crash*           |
            | 9     | Unknown/Not Reported           |

Las heridas con grado de severidad 4 indican que la persona sufrió heridas que le provocaron la muerte.

In [ ]:
np.sort(pr['inj_sev'].unique())

array([0, 1, 2, 3, 4, 5, 6, 9])

In [ ]:
pr.groupby(['inj_sev'])[['accident_id']].count()

,accident_id
inj_sev,
0,2908
1,681
2,1334
3,1135
4,4302
5,67
6,2
9,182


La tabla de personas registra que 4302 personas perdieron la vida en accidentes de tránsito en Arizona. Comparemos con el total que hay en la columna 'fatals' de la tabla de accidentes.

In [ ]:
acc['fatals'].sum()

4302

El total de muertos coincide, por tanto, podemos usar la columna 'inj_sev' para determinar si una persona murió a causa del accidente.

Renombremos la categoría 'Unknown/Not Reported' a solo 'Not reported y asignemos el nombre de la categoría a su respectivo valor.

In [ ]:
dic_inj_sev = {
    0: 'No Apparent Injury (O)',
    1: 'Possible Injury (C)',
    2: 'Suspected Minor Injury (B)',
    3: 'Suspected Serious Injury (A)',
    4: 'Fatal Injury (K)',
    5: 'Injured, Severity Unknown',
    6: 'Died Prior to Crash*',
    9: 'Not Reported'
}
pr['inj_sev'] = pr['inj_sev'].replace(dic_inj_sev)

<ipython-input-767-9ab26148ec2a>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.groupby(['inj_sev'])[['accident_id']].count()

,accident_id
inj_sev,
Died Prior to Crash*,2
Fatal Injury (K),4302
"Injured, Severity Unknown",67
No Apparent Injury (O),2908
Not Reported,182
Possible Injury (C),681
Suspected Minor Injury (B),1334
Suspected Serious Injury (A),1135


- **drinking:** La policía reporta que la persona se encontraba involucrada con alcohol al momento del accidente.

        | Codes | Attributes                      |
        |-------|---------------------------------|
        | 0     | No (Alcohol Not Involved)       |
        | 1     | Yes (Alcohol Involved)          |
        | 8     | Not Reported                    |
        | 9     | Unknown (Police Reported)       |

- **alc_res:** Resultado de la prueba de alcoholemia a la persona.

        | CRSS Codes | FARS Codes | Attributes                             |
        |------------|------------|----------------------------------------|
        | 000-939    | 000-939    | Actual Value                           |
        | 940        | 940        | .94 or Greater                         |
        | 996        | 996        | Test Not Given                         |
        | 997        | 997        | AC Test Performed, Results Unknown     |
        | 998        | 998        | Positive Reading with No Actual Value  |
        | 995        | 995        | Not Reported                           |
        | 999        | 999        | Unknown if Tested                      |

In [ ]:
pr['drinking'].unique()

array([1, 8, 9, 0])

In [ ]:
pr.groupby(['drinking'])[['accident_id']].count()

,accident_id
drinking,
0,3958
1,975
8,3618
9,2060


Solo la categoría 998 (Positive Reading with No Actual Value) será tratada como nulo, ya que indica que el resultado es positivo, pero no fue una prueba que arroja resultados numéricos, sino de tipo binario. Más delante se tratarán estos valores. Las categorías 996, 997, 995 y 999 serán reemplazadas por ceros.

In [ ]:
pr['alc_res'] = pr['alc_res'].replace({998 : np.nan})

<ipython-input-771-12629178582a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr['alc_res'] = pr['alc_res'].replace([996,997,995,999], 0)

<ipython-input-772-40e706cf96bc>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
np.sort(pr['drinking'].unique())

array([0, 1, 8, 9])

- **drugs:** La policia reporta que la persona se encontraba involucrada con drogas al momento del accidente.

        | Codes | Attributes                     |
        |-------|--------------------------------|
        | 0     | No (Drugs Not Involved)        |
        | 1     | Yes (Drugs Involved)           |
        | 8     | Not Reported                   |
        | 9     | Unknown (Police Reported)      |



- **drugres1, drugres2, drugres3:** Resultado de los diferentes test de drogas.

        | CRSS Codes | FARS Codes | Attributes                                           |
        |------------|------------|------------------------------------------------------|
        | 000        | 000        | Test Not Given                                       |
        | 001        | 001        | Tested, No Drugs Found/Negative                      |
        | XXX        | 100-295    | Narcotic*                                            |
        | XXX        | 300-399    | Depressant*                                          |
        | XXX        | 400-495    | Stimulant*                                           |
        | XXX        | 500-595    | Hallucinogen*                                        |
        | XXX        | 600-695    | Cannabinoid*                                         |
        | XXX        | 700-795    | Phencyclidine (PCP)*                                 |
        | XXX        | 800-895    | Anabolic Steroid*                                    |
        | XXX        | 900-995    | Inhalant*                                            |
        | XXX        | 996        | Other Drug                                           |
        | 997        | 997        | Tested for Drugs, Results Unknown                    |
        | 998        | 998        | Tested for Drugs, Drugs Found, Type Unknown/Positive |
        | 095        | 095        | Not Reported                                         |
        | 999        | 999        | Unknown If Tested                                    |

Para esta variable usaremos las categorías tal cual están.

In [ ]:
np.sort(pr['drugs'].unique())

array([0, 1, 8, 9])

In [ ]:
pr.groupby(['drugs'])[['accident_id']].count()

,accident_id
drugs,
0,4451
1,312
8,3769
9,2079


In [ ]:
for col in ['drugres1', 'drugres2', 'drugres3']:
    pr[col] = pr[col].apply(lambda x: 'Test Not Given' if x==0
                        else 'Tested, No Drugs Found/Negative' if x==1
                        else 'Narcotic' if x>=100 and x<=295
                        else 'Depressant' if x>=300 and x<=399
                        else 'Stimulant' if x>=400 and x<=495
                        else 'Hallucinogen' if x>=500 and x<=595
                        else 'Cannabinoid' if x>=600 and x<=695
                        else 'Phencyclidine (PCP)' if x>=700 and x<=795
                        else 'Anabolic Steroid' if x>=800 and x<=895
                        else 'Inhalant' if x>=900 and x<=995
                        else 'Other Drug' if x==996
                        else 'Tested for Drugs, Results Unknown' if x==997
                        else 'Tested for Drugs, Drugs Found, Type Unknown/Positive' if x==998
                        else 'Not Reported' if x==95
                        else 'Unknown If Tested' if x==999
                        else x)

<ipython-input-776-1135ea111020>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.groupby(['drugres1'])[['accident_id']].count()

,accident_id
drugres1,
Cannabinoid,216
Depressant,233
Hallucinogen,7
Narcotic,235
Not Reported,88
Other Drug,292
Phencyclidine (PCP),3
Stimulant,328
Test Not Given,7020


# 4. Tratamiento de datos nulos

## 4.1. Tabla accidentes

In [ ]:
acc.isnull().sum()

accident_id       0
county_name       6
day               0
month             0
year              0
hour            118
minute          118
latitude        271
longitud        271
sp_jur_lit        0
man_coll_lit      0
lgt_cond_lit      0
weather_lit       0
fatals            0
dtype: int64

### 4.1.1. county_name

In [ ]:
acc[acc['county_name'].isnull()]

,accident_id,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals
733,2012040750,NaN,31,10,2012,NaN,NaN,NaN,NaN,Unknown,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
1069,2013040342,NaN,7,5,2013,NaN,NaN,NaN,NaN,Indian Reservation,Other,Not Reported,Not Reported,1
1491,2013040784,NaN,13,12,2013,NaN,NaN,NaN,NaN,Unknown,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
2949,2015040749,NaN,7,11,2015,NaN,NaN,NaN,NaN,Unknown,Other,Not Reported,Not Reported,1
3026,2015040830,NaN,25,7,2015,NaN,NaN,NaN,NaN,Unknown,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1
3281,2016040248,NaN,1,4,2016,12.0,30.0,NaN,NaN,Unknown,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1


Al ser solo 6 nulos y que, además, dichos registros no tienen latitud y longitud para poder obtener el condado, decidimos eliminar estos 6 registros.

In [ ]:
acc = acc.dropna(subset=['county_name'])

### 4.1.2. hour y minute

Vamos a usar la moda para rellenar los valores nulos en la hora y el minuto en el que ocurrió el accidente.

In [ ]:
acc['hour'] = acc['hour'].fillna(acc['hour'].mode()[0])
acc['minute'] = acc['minute'].fillna(acc['minute'].mode()[0])

### 4.1.3. latitude y longitud

Para los nulos de las columnas "latitude" y "longitud" asignaremos valores aleatorios de los otros accidentes pertenecientes al mismo condado.

Utilizando la función choice() de la librería random, creamos una función que seleccione estos valores aleatorios. Como son valores aleatorios, estarían cambiando cada que se ejecute el código, por ellos, realizamos una ejecución y guardamos esos valores.

        import random as rnd
        def sel_lat_long(row, df):
            valores = df[(df['county_name'] == row['county_name'])][['latitude', 'longitud']].values
            aleatorios = rnd.choice(valores)
            row['latitude'], row['longitud'] = aleatorios[0], aleatorios[1]
            return row

        acc_no_nulos = acc[(acc['latitude'].notnull()) & (acc['longitud'].notnull())]

        acc_lat_lon = acc[(acc['latitude'].isnull()) & (acc['longitud'].isnull())].apply(lambda x:  sel_lat_long(x, acc_no_nulos), axis=1)
        latitudes = acc_lat_lon['latitude'].values
        longitudes = acc_lat_lon['longitud'].values

        latitudes: [32.24272778, 35.16088333, 36.3998, 34.25295, 33.49081667, 36.41089722, 35.77568056, 32.20629444, 36.56006944, 36.41317222, 32.16275278, 35.16799722, 33.38076944, 34.90769722, 35.04634444, 32.09219444, 33.62906944, 32.786675, 35.09975556, 36.42489444, 33.24977778, 34.08328056, 36.79376667, 32.137175, 34.48534722, 34.47005, 33.15205833, 33.74828611, 33.83512222, 33.69831667, 33.45135833, 35.75955, 36.80573889, 32.85004444, 33.66698611, 36.369625, 35.02789722, 35.66908056, 34.34005556, 36.21508889, 34.93873611, 34.27523889, 36.77855556, 33.37926389, 33.71282222, 34.488275, 35.21636944, 33.58835, 33.28974722, 33.45252778, 34.86684444, 34.8416, 34.07906111, 34.02711667, 35.25708889, 36.72498611, 32.19701667, 35.17908333, 36.42489444, 33.04832778, 32.89429167, 36.49790278, 34.47086389, 35.199075, 34.44590278, 34.82013056, 34.023, 32.33693056, 32.95574167, 32.3125, 33.78133333, 31.33914167, 33.34836944, 33.17945833, 35.19382222, 33.46373611, 33.50003889, 33.43221389, 32.70803611, 36.80573889, 34.34819167, 35.70535278, 36.15131111, 36.66825833, 36.40795556, 33.62002222, 34.91563611, 36.71561667, 36.84975833, 33.45128889, 36.73248611, 36.12125833, 33.79907778, 35.68562778, 34.37009444, 34.21860278, 34.19603889, 35.10061111, 33.33071944, 34.47380278, 36.74848056, 35.00672222, 33.64043889, 36.16675, 33.38686389, 33.26188889, 35.48035278, 33.49771667, 32.22149444, 33.28676667, 36.66825833, 33.38659722, 33.34390556, 33.59788333, 33.13222778, 32.18561111, 32.5328, 34.28596111, 33.59628889, 36.62705833, 33.385875, 33.55524444, 32.20677222, 33.41521389, 36.46117222, 33.46211944, 33.56755278, 34.83660833, 33.68198611, 33.94049167, 33.21849444, 33.45956944, 33.48958889, 32.63569722, 36.46117222, 36.07062778, 34.32237778, 32.395375, 35.184075, 33.40188611, 32.32525, 32.69944722, 33.21901389, 33.39324167, 33.90950556, 32.17758611, 32.01261389, 35.25581667, 35.56382222, 34.38087778, 33.28779722, 34.05880278, 33.30218333, 32.81631389, 33.51838611, 36.33086389, 33.38985278, 33.37774722, 33.40736389, 34.99800556, 33.62606944, 33.99546111, 33.71375278, 33.16193889, 33.92496111, 33.74069722, 33.66983889, 36.59364167, 35.69533611, 33.42874444, 33.88346111, 35.19249444, 33.45032222, 34.37953333, 33.28298889, 32.683925, 35.064075, 34.27523333, 33.52370833, 36.10034722, 33.46751111, 33.49506389, 33.64856667, 33.38875556, 33.46163056, 33.66033056, 32.82123611, 36.40795556, 33.45479167, 33.46215, 33.46596944, 33.451875, 33.631975, 33.55036944, 33.37768889, 33.5099, 35.03886389, 33.45141944, 33.38673889, 33.39955833, 33.48403333, 33.42918889, 34.41833333, 33.62276111, 33.440575, 33.37826111, 34.80336389, 32.71273611, 33.50095833, 34.19189722, 35.21113333, 33.25208333, 33.46373611, 33.39220556, 33.39228333, 33.63796111, 33.24830556, 33.21881111, 35.18169444, 33.66724167, 33.63455556, 33.33387778, 33.41845278, 33.35570278, 33.91348333, 35.28053889, 35.84017778, 32.93644444, 33.39371944, 33.56834167, 33.62373889, 35.78288333, 35.71020556, 33.67562222, 33.60165833, 33.48768056, 33.88299722, 33.34701944, 33.37839722, 33.60915, 31.98536389, 33.42215, 33.55206389, 33.38593056, 33.56350833, 33.47047778, 33.48853056, 33.49190556, 34.21662222, 35.21188333, 36.33086389, 32.40115556, 33.59638889, 33.44731667, 33.63663333, 34.25429444, 33.28875833, 33.28972778, 33.64074722, 31.69246389, 33.34771944, 33.34915, 33.48778611, 34.98073333, 33.37926389]

        longitudes: [-110.80294444, -111.28165   , -109.74735278, -110.03553889, -112.11706389, -110.82730833, -109.73328333, -110.940525  , -110.74889722, -109.27279167, -110.95612778, -111.66717222, -111.89224722, -109.308675  , -109.80605556, -110.99311389, -112.11661389, -113.56771667, -109.63806667, -109.1998    , -111.97768056, -109.906375  , -109.94148889, -110.00358333, -110.24477778, -114.35227778, -111.86853611, -111.99510278, -109.96435556, -112.352775  , -111.87502222, -109.06401389, -110.23286667, -111.911925  , -113.66811944, -110.89614167, -109.83758889, -109.16520833, -110.72540833, -109.60281944, -110.12941111, -109.77793056, -110.03285278, -111.72240278, -112.141525  , -110.08223333, -111.58216111, -113.47974722, -111.97160278, -112.13451944, -111.76782778, -114.14541111, -109.89128056, -109.90985   , -109.29092778, -110.13483889, -110.82393056, -111.63893611, -109.1998    , -109.320725  , -111.78446389, -110.62170833, -110.00945278, -111.58966944, -111.39188056, -110.96233056, -109.90530556, -110.96959444, -111.71515556, -110.978175  , -109.18025556, -110.95422222, -112.16909722, -111.03959722, -109.41930556, -112.34088056, -112.134225  , -112.20116944, -111.48647222, -110.23286667, -111.425675  , -109.30896667, -109.54645833, -109.59835556, -109.60799167, -112.16842222, -110.82848056, -111.59224167, -109.29566667, -112.1604    , -109.63265   , -109.630625  , -109.93785   , -109.23845556, -109.99098889, -110.07244722, -109.99022778, -114.59799722, -111.63546667, -114.29141389, -110.22663611, -110.62311111, -112.05333333, -109.5855    , -112.16837222, -111.95292778, -109.52936111, -112.20322222, -110.90969444, -111.84161111, -109.59835556, -111.67265833, -111.83632778, -112.12966389, -111.86979167, -110.96096111, -110.86882222, -111.33846667, -112.19030278, -109.58377778, -111.84636111, -112.04464722, -110.977275  , -111.62400556, -110.69211667, -112.05811389, -112.171275  , -111.88853611, -111.982325  , -109.93449167, -111.84453333, -112.03758889, -111.926175  , -111.37687778, -110.69211667, -109.61056389, -109.75814167, -111.15076111, -111.66850556, -111.82296944, -111.14411944, -111.47269722, -111.89021389, -111.51249167, -109.94344722, -112.26990833, -110.9922    , -111.92266111, -111.90958056, -110.60205556, -111.84167222, -109.49805833, -111.97245278, -109.48746111, -112.16211389, -109.6221    , -111.82290833, -112.06935   , -111.64094722, -110.59291944, -111.96033611, -109.12428611, -110.53478056, -111.49633889, -111.44908611, -112.48076667, -112.16288611, -111.16138056, -109.27210833, -112.30640556, -112.65836111, -110.79804167, -112.72601389, -111.34480556, -111.77743611, -114.63019444, -109.2824    , -109.77792222, -112.04626111, -111.19578333, -112.01298611, -112.18644444, -112.070075  , -111.69393611, -112.15782222, -114.24019167, -111.56679444, -109.60799167, -112.74590556, -112.02360278, -112.164375  , -111.70150833, -112.116125  , -112.18615833, -112.12636667, -112.151175  , -110.72554444, -112.13546389, -111.89125278, -112.07095556, -112.0216    , -112.34944167, -110.65683889, -111.92340556, -112.13455833, -111.92578333, -110.20802222, -113.82592222, -112.25505833, -110.09105   , -111.60746944, -111.96363333, -112.34088056, -112.11540556, -112.0574    , -112.28983889, -111.78946667, -111.82553889, -111.65269444, -112.21931389, -111.89094167, -112.17492222, -111.89145   , -112.62618056, -113.99158056, -109.18771389, -109.60186667, -112.77308333, -111.74953333, -112.22225833, -112.35597222, -109.71780278, -109.52184722, -112.13468333, -111.99164167, -112.15166667, -112.65769444, -112.08458056, -112.39728889, -112.27016944, -110.68108333, -111.87726944, -112.11193333, -111.96793611, -112.09974444, -112.1517    , -112.16891389, -112.11702778, -111.46553056, -111.77485556, -109.6221    , -111.13989167, -112.13396667, -111.82244444, -111.91864167, -110.02895   , -111.69203611, -112.798125  , -112.00309444, -110.36376944, -111.59244167, -111.9659    , -112.01727778, -110.05339444, -111.705125  ]

In [ ]:
nulos_lat_long = acc[(acc['latitude'].isnull()) & (acc['longitud'].isnull())].index

In [ ]:
acc.loc[nulos_lat_long, 'latitude'] = [32.24272778, 35.16088333, 36.3998, 34.25295, 33.49081667, 36.41089722, 35.77568056, 32.20629444, 36.56006944, 36.41317222, 32.16275278, 35.16799722, 33.38076944, 34.90769722, 35.04634444, 32.09219444, 33.62906944, 32.786675, 35.09975556, 36.42489444, 33.24977778, 34.08328056, 36.79376667, 32.137175, 34.48534722, 34.47005, 33.15205833, 33.74828611, 33.83512222, 33.69831667, 33.45135833, 35.75955, 36.80573889, 32.85004444, 33.66698611, 36.369625, 35.02789722, 35.66908056, 34.34005556, 36.21508889, 34.93873611, 34.27523889, 36.77855556, 33.37926389, 33.71282222, 34.488275, 35.21636944, 33.58835, 33.28974722, 33.45252778, 34.86684444, 34.8416, 34.07906111, 34.02711667, 35.25708889, 36.72498611, 32.19701667, 35.17908333, 36.42489444, 33.04832778, 32.89429167, 36.49790278, 34.47086389, 35.199075, 34.44590278, 34.82013056, 34.023, 32.33693056, 32.95574167, 32.3125, 33.78133333, 31.33914167, 33.34836944, 33.17945833, 35.19382222, 33.46373611, 33.50003889, 33.43221389, 32.70803611, 36.80573889, 34.34819167, 35.70535278, 36.15131111, 36.66825833, 36.40795556, 33.62002222, 34.91563611, 36.71561667, 36.84975833, 33.45128889, 36.73248611, 36.12125833, 33.79907778, 35.68562778, 34.37009444, 34.21860278, 34.19603889, 35.10061111, 33.33071944, 34.47380278, 36.74848056, 35.00672222, 33.64043889, 36.16675, 33.38686389, 33.26188889, 35.48035278, 33.49771667, 32.22149444, 33.28676667, 36.66825833, 33.38659722, 33.34390556, 33.59788333, 33.13222778, 32.18561111, 32.5328, 34.28596111, 33.59628889, 36.62705833, 33.385875, 33.55524444, 32.20677222, 33.41521389, 36.46117222, 33.46211944, 33.56755278, 34.83660833, 33.68198611, 33.94049167, 33.21849444, 33.45956944, 33.48958889, 32.63569722, 36.46117222, 36.07062778, 34.32237778, 32.395375, 35.184075, 33.40188611, 32.32525, 32.69944722, 33.21901389, 33.39324167, 33.90950556, 32.17758611, 32.01261389, 35.25581667, 35.56382222, 34.38087778, 33.28779722, 34.05880278, 33.30218333, 32.81631389, 33.51838611, 36.33086389, 33.38985278, 33.37774722, 33.40736389, 34.99800556, 33.62606944, 33.99546111, 33.71375278, 33.16193889, 33.92496111, 33.74069722, 33.66983889, 36.59364167, 35.69533611, 33.42874444, 33.88346111, 35.19249444, 33.45032222, 34.37953333, 33.28298889, 32.683925, 35.064075, 34.27523333, 33.52370833, 36.10034722, 33.46751111, 33.49506389, 33.64856667, 33.38875556, 33.46163056, 33.66033056, 32.82123611, 36.40795556, 33.45479167, 33.46215, 33.46596944, 33.451875, 33.631975, 33.55036944, 33.37768889, 33.5099, 35.03886389, 33.45141944, 33.38673889, 33.39955833, 33.48403333, 33.42918889, 34.41833333, 33.62276111, 33.440575, 33.37826111, 34.80336389, 32.71273611, 33.50095833, 34.19189722, 35.21113333, 33.25208333, 33.46373611, 33.39220556, 33.39228333, 33.63796111, 33.24830556, 33.21881111, 35.18169444, 33.66724167, 33.63455556, 33.33387778, 33.41845278, 33.35570278, 33.91348333, 35.28053889, 35.84017778, 32.93644444, 33.39371944, 33.56834167, 33.62373889, 35.78288333, 35.71020556, 33.67562222, 33.60165833, 33.48768056, 33.88299722, 33.34701944, 33.37839722, 33.60915, 31.98536389, 33.42215, 33.55206389, 33.38593056, 33.56350833, 33.47047778, 33.48853056, 33.49190556, 34.21662222, 35.21188333, 36.33086389, 32.40115556, 33.59638889, 33.44731667, 33.63663333, 34.25429444, 33.28875833, 33.28972778, 33.64074722, 31.69246389, 33.34771944, 33.34915, 33.48778611, 34.98073333, 33.37926389]
acc.loc[nulos_lat_long, 'longitud'] = [-110.80294444, -111.28165   , -109.74735278, -110.03553889, -112.11706389, -110.82730833, -109.73328333, -110.940525  , -110.74889722, -109.27279167, -110.95612778, -111.66717222, -111.89224722, -109.308675  , -109.80605556, -110.99311389, -112.11661389, -113.56771667, -109.63806667, -109.1998    , -111.97768056, -109.906375  , -109.94148889, -110.00358333, -110.24477778, -114.35227778, -111.86853611, -111.99510278, -109.96435556, -112.352775  , -111.87502222, -109.06401389, -110.23286667, -111.911925  , -113.66811944, -110.89614167, -109.83758889, -109.16520833, -110.72540833, -109.60281944, -110.12941111, -109.77793056, -110.03285278, -111.72240278, -112.141525  , -110.08223333, -111.58216111, -113.47974722, -111.97160278, -112.13451944, -111.76782778, -114.14541111, -109.89128056, -109.90985   , -109.29092778, -110.13483889, -110.82393056, -111.63893611, -109.1998    , -109.320725  , -111.78446389, -110.62170833, -110.00945278, -111.58966944, -111.39188056, -110.96233056, -109.90530556, -110.96959444, -111.71515556, -110.978175  , -109.18025556, -110.95422222, -112.16909722, -111.03959722, -109.41930556, -112.34088056, -112.134225  , -112.20116944, -111.48647222, -110.23286667, -111.425675  , -109.30896667, -109.54645833, -109.59835556, -109.60799167, -112.16842222, -110.82848056, -111.59224167, -109.29566667, -112.1604    , -109.63265   , -109.630625  , -109.93785   , -109.23845556, -109.99098889, -110.07244722, -109.99022778, -114.59799722, -111.63546667, -114.29141389, -110.22663611, -110.62311111, -112.05333333, -109.5855    , -112.16837222, -111.95292778, -109.52936111, -112.20322222, -110.90969444, -111.84161111, -109.59835556, -111.67265833, -111.83632778, -112.12966389, -111.86979167, -110.96096111, -110.86882222, -111.33846667, -112.19030278, -109.58377778, -111.84636111, -112.04464722, -110.977275  , -111.62400556, -110.69211667, -112.05811389, -112.171275  , -111.88853611, -111.982325  , -109.93449167, -111.84453333, -112.03758889, -111.926175  , -111.37687778, -110.69211667, -109.61056389, -109.75814167, -111.15076111, -111.66850556, -111.82296944, -111.14411944, -111.47269722, -111.89021389, -111.51249167, -109.94344722, -112.26990833, -110.9922    , -111.92266111, -111.90958056, -110.60205556, -111.84167222, -109.49805833, -111.97245278, -109.48746111, -112.16211389, -109.6221    , -111.82290833, -112.06935   , -111.64094722, -110.59291944, -111.96033611, -109.12428611, -110.53478056, -111.49633889, -111.44908611, -112.48076667, -112.16288611, -111.16138056, -109.27210833, -112.30640556, -112.65836111, -110.79804167, -112.72601389, -111.34480556, -111.77743611, -114.63019444, -109.2824    , -109.77792222, -112.04626111, -111.19578333, -112.01298611, -112.18644444, -112.070075  , -111.69393611, -112.15782222, -114.24019167, -111.56679444, -109.60799167, -112.74590556, -112.02360278, -112.164375  , -111.70150833, -112.116125  , -112.18615833, -112.12636667, -112.151175  , -110.72554444, -112.13546389, -111.89125278, -112.07095556, -112.0216    , -112.34944167, -110.65683889, -111.92340556, -112.13455833, -111.92578333, -110.20802222, -113.82592222, -112.25505833, -110.09105   , -111.60746944, -111.96363333, -112.34088056, -112.11540556, -112.0574    , -112.28983889, -111.78946667, -111.82553889, -111.65269444, -112.21931389, -111.89094167, -112.17492222, -111.89145   , -112.62618056, -113.99158056, -109.18771389, -109.60186667, -112.77308333, -111.74953333, -112.22225833, -112.35597222, -109.71780278, -109.52184722, -112.13468333, -111.99164167, -112.15166667, -112.65769444, -112.08458056, -112.39728889, -112.27016944, -110.68108333, -111.87726944, -112.11193333, -111.96793611, -112.09974444, -112.1517    , -112.16891389, -112.11702778, -111.46553056, -111.77485556, -109.6221    , -111.13989167, -112.13396667, -111.82244444, -111.91864167, -110.02895   , -111.69203611, -112.798125  , -112.00309444, -110.36376944, -111.59244167, -111.9659    , -112.01727778, -110.05339444, -111.705125  ]

In [ ]:
acc.isnull().sum()

accident_id     0
county_name     0
day             0
month           0
year            0
hour            0
minute          0
latitude        0
longitud        0
sp_jur_lit      0
man_coll_lit    0
lgt_cond_lit    0
weather_lit     0
fatals          0
dtype: int64

## 4.2. Tabla vehicle

In [ ]:
vh.isnull().sum()

accident_id      0
mod_year       409
veh_no           0
make             0
dtype: int64

In [ ]:
np.sort(vh['mod_year'].unique())

array([1932., 1956., 1957., 1963., 1965., 1966., 1967., 1968., 1969.,
       1970., 1971., 1972., 1973., 1974., 1976., 1977., 1978., 1979.,
       1980., 1981., 1982., 1983., 1984., 1985., 1986., 1987., 1988.,
       1989., 1990., 1991., 1992., 1993., 1994., 1995., 1996., 1997.,
       1998., 1999., 2000., 2001., 2002., 2003., 2004., 2005., 2006.,
       2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014., 2015.,
       2016., 2017.,   nan])

A los valores nulos de la columna "mod_year" asignamos la moda de los años desde el año en que ocurrió el accidente hacia atrás. Para ello, hay que agregar la columna "year".

In [ ]:
vh['year'] = df_vh['Year']

<ipython-input-787-5349758858ba>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
nulos_vh = vh[vh['mod_year'].isnull()].index
vh.loc[nulos_vh, 'mod_year'] = vh.iloc[nulos_vh].apply(lambda x: vh.loc[vh['mod_year']<=x['year'], 'mod_year'].mode(), axis=1)[0]

In [ ]:
vh.isnull().sum()

accident_id    0
mod_year       0
veh_no         0
make           0
year           0
dtype: int64

In [ ]:
vh.drop('year', axis=1, inplace=True)

<ipython-input-790-9116a69acefd>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 4.3. Tabla person

In [ ]:
pr.isnull().sum()

accident_id    0
age            0
sex            0
inj_sev        0
drinking       0
alc_res        5
drugs          0
drugres1       0
drugres2       0
drugres3       0
air_bag        0
veh_no         0
dtype: int64

Como vimos en la depuración de la tabla "person", estos nulos son personas con resultado positivo en prueba de alcoholemia. Por tanto, pondremos el promedio de la columna "alc_res".

In [ ]:
pr['alc_res'].fillna(pr['alc_res'].mean(), inplace=True)

<ipython-input-792-033750ae8942>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**air_bag**

        | Codes  | Attributes                             |
        |--------|----------------------------------------|
        | 00     | Not Applicable                         |
        | 01     | Deployed-Front                         |
        | 02     | Deployed-Side (door, seatback)         |
        | 03     | Deployed-Curtain (roof)                |
        | 07     | Deployed-Other (knee, air belt, etc.)  |
        | 08     | Deployed-Combination                   |
        | 09     | Deployment-Unknown Location            |
        | 20     | Not Deployed                           |
        | 28     | Switched Off                           |
        | 98     | Not Reported                           |
        | 99     | Deployment Unknown                     |


In [ ]:
pr['air_bag'].unique()

array([ 1,  0, 97, 99, 20,  9,  8, 98,  3,  2,  7])

In [ ]:
dic_air_bag = {
    0: 'Not Applicable',
    1: 'Deployed-Front',
    2: 'Deployed-Side (door, seatback)',
    3: 'Deployed-Curtain (roof)',
    7: 'Deployed-Other (knee, air belt, etc.)',
    8: 'Deployed-Combination',
    9: 'Deployment-Unknown Location',
    20: 'Not Deployed',
    28: 'Switched Off',
    97: 'Deployment Unknown',
    98: 'Not Reported',
    99: 'Deployment Unknown'
}

pr['air_bag'] = pr['air_bag'].replace(dic_air_bag)

<ipython-input-794-9f166e2c9d17>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.groupby(['air_bag'])[['accident_id']].count().reset_index().rename(columns={'accident_id':'accidentes'})

,air_bag,accidentes
0,Deployed-Combination,161
1,Deployed-Curtain (roof),44
2,Deployed-Front,790
3,"Deployed-Other (knee, air belt, etc.)",8
4,"Deployed-Side (door, seatback)",22
5,Deployment Unknown,1952
6,Deployment-Unknown Location,451
7,Not Applicable,3317
8,Not Deployed,3824
9,Not Reported,42


In [ ]:
pr.isnull().sum()

accident_id    0
age            0
sex            0
inj_sev        0
drinking       0
alc_res        0
drugs          0
drugres1       0
drugres2       0
drugres3       0
air_bag        0
veh_no         0
dtype: int64

**age**

In [ ]:
pr[(pr['age']==998) | (pr['age']==999)]

,accident_id,age,sex,inj_sev,drinking,alc_res,drugs,drugres1,drugres2,drugres3,air_bag,veh_no
6,2016040002,999,Not Reported,No Apparent Injury (O),9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1
72,2016040026,999,Not Reported,No Apparent Injury (O),9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1
91,2016040032,999,Female,Possible Injury (C),8,0.0,8,Test Not Given,Test Not Given,Test Not Given,Not Reported,2
148,2016040055,999,Not Reported,Not Reported,9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,2
191,2016040074,999,Not Reported,Not Reported,9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1
226,2016040087,999,Not Reported,Not Reported,9,0.0,9,Unknown If Tested,Test Not Given,Test Not Given,Deployment Unknown,1
234,2016040089,999,Female,"Injured, Severity Unknown",9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,2
235,2016040089,999,Female,"Injured, Severity Unknown",8,0.0,8,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,2
276,2016040103,999,Not Reported,No Apparent Injury (O),9,0.0,9,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1
284,2016040107,999,Female,Suspected Serious Injury (A),0,0.0,0,Test Not Given,Test Not Given,Test Not Given,Not Deployed,1


Realizando una inspección a los registros con nulos en la columa "age", se observa que estos no tienen información de valor en las demás columnas, por tanto, se decide eliminarlos.

In [ ]:
pr['age'] = pr['age'].replace([998, 999], np.nan)

<ipython-input-798-82d8f491d616>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.dropna(inplace=True)

<ipython-input-799-2850b0996c3c>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
pr.isnull().sum()

accident_id    0
age            0
sex            0
inj_sev        0
drinking       0
alc_res        0
drugs          0
drugres1       0
drugres2       0
drugres3       0
air_bag        0
veh_no         0
dtype: int64

# 5. Generación de bodegas de datos

## 5.1. Se unen las 3 bases de datos en una sola con ''accident_id'' como llave que une las 3

In [ ]:
acc.shape, vh.shape, pr.shape

((3898, 14), (5889, 4), (10185, 12))

In [ ]:
pr_fn = pr.merge(vh, how='outer', on=['accident_id', 'veh_no']).merge(acc, how='outer', on=['accident_id'])
pr_fn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10416 entries, 0 to 10415
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   accident_id   10416 non-null  int64  
 1   age           10185 non-null  float64
 2   sex           10185 non-null  object 
 3   inj_sev       10185 non-null  object 
 4   drinking      10185 non-null  float64
 5   alc_res       10185 non-null  float64
 6   drugs         10185 non-null  float64
 7   drugres1      10185 non-null  object 
 8   drugres2      10185 non-null  object 
 9   drugres3      10185 non-null  object 
 10  air_bag       10185 non-null  object 
 11  veh_no        10416 non-null  int64  
 12  mod_year      9407 non-null   float64
 13  make          9407 non-null   float64
 14  county_name   10409 non-null  object 
 15  day           10409 non-null  float64
 16  month         10409 non-null  float64
 17  year          10409 non-null  float64
 18  hour          10409 non-nu

In [ ]:
pr_fn['mod_year'].fillna('Not vehicle', inplace=True)

In [ ]:
pr_fn.isnull().sum()

accident_id        0
age              231
sex              231
inj_sev          231
drinking         231
alc_res          231
drugs            231
drugres1         231
drugres2         231
drugres3         231
air_bag          231
veh_no             0
mod_year           0
make            1009
county_name        7
day                7
month              7
year               7
hour               7
minute             7
latitude           7
longitud           7
sp_jur_lit         7
man_coll_lit       7
lgt_cond_lit       7
weather_lit        7
fatals             7
dtype: int64

Eliminamos los registros con nulos, dado que son muy pocos.

In [ ]:
pr_fn.dropna(inplace=True)

Buscar y eliminar duplicados.

In [ ]:
pr_fn[pr_fn.duplicated()]

,accident_id,age,sex,inj_sev,drinking,alc_res,drugs,drugres1,drugres2,drugres3,air_bag,veh_no,mod_year,make,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals
726,2016040287,11.0,Male,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2002.0,37.0,MOHAVE,8.0,5.0,2016.0,5.0,29.0,36.820986,-114.038167,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dawn,Cloudy,1.0
945,2016040373,18.0,Male,Suspected Serious Injury (A),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,3,2011.0,37.0,LA PAZ,17.0,6.0,2016.0,3.0,11.0,33.654983,-113.956861,No Special Jurisdiction,Front-to-Rear,Dark - Not Lighted,Clear,2.0
975,2016040381,13.0,Male,Fatal Injury (K),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,2,2001.0,12.0,COCONINO,27.0,6.0,2016.0,10.0,40.0,36.612294,-111.072586,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,2.0
1154,2016040435,18.0,Male,Suspected Serious Injury (A),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2002.0,49.0,MARICOPA,10.0,7.0,2016.0,3.0,0.0,33.461342,-112.110506,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dark - Lighted,Clear,1.0
1212,2016040456,52.0,Female,Fatal Injury (K),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1,2016.0,7.0,MOHAVE,24.0,7.0,2016.0,13.0,40.0,35.534200,-114.355447,No Special Jurisdiction,Angle,Daylight,Clear,4.0
1305,2016040488,2.0,Female,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,2,2002.0,12.0,NAVAJO,1.0,8.0,2016.0,13.0,53.0,36.461172,-110.692117,Indian Reservation,Angle,Daylight,Clear,1.0
1308,2016040488,6.0,Female,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,2,2002.0,12.0,NAVAJO,1.0,8.0,2016.0,13.0,53.0,36.461172,-110.692117,Indian Reservation,Angle,Daylight,Clear,1.0
1396,2016040522,15.0,Female,Possible Injury (C),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2011.0,23.0,MARICOPA,6.0,8.0,2016.0,17.0,3.0,33.582836,-111.618108,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0
1397,2016040522,15.0,Female,Possible Injury (C),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2011.0,23.0,MARICOPA,6.0,8.0,2016.0,17.0,3.0,33.582836,-111.618108,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0
1398,2016040522,15.0,Female,Possible Injury (C),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2011.0,23.0,MARICOPA,6.0,8.0,2016.0,17.0,3.0,33.582836,-111.618108,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0


In [ ]:
pr_fn.drop_duplicates(inplace=True)

In [ ]:
pr_fn[pr_fn.duplicated()]

,accident_id,age,sex,inj_sev,drinking,alc_res,drugs,drugres1,drugres2,drugres3,air_bag,veh_no,mod_year,make,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals


In [ ]:
pr_fn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9118 entries, 0 to 10407
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   accident_id   9118 non-null   int64  
 1   age           9118 non-null   float64
 2   sex           9118 non-null   object 
 3   inj_sev       9118 non-null   object 
 4   drinking      9118 non-null   float64
 5   alc_res       9118 non-null   float64
 6   drugs         9118 non-null   float64
 7   drugres1      9118 non-null   object 
 8   drugres2      9118 non-null   object 
 9   drugres3      9118 non-null   object 
 10  air_bag       9118 non-null   object 
 11  veh_no        9118 non-null   int64  
 12  mod_year      9118 non-null   object 
 13  make          9118 non-null   float64
 14  county_name   9118 non-null   object 
 15  day           9118 non-null   float64
 16  month         9118 non-null   float64
 17  year          9118 non-null   float64
 18  hour          9118 non-null

## 5.2. Unión entre accidentes y vehículos (vehículos final)

In [ ]:
vh_fn = acc.merge(vh, how='outer', on='accident_id')
vh_fn.isnull().sum()

accident_id     0
county_name     6
day             6
month           6
year            6
hour            6
minute          6
latitude        6
longitud        6
sp_jur_lit      6
man_coll_lit    6
lgt_cond_lit    6
weather_lit     6
fatals          6
mod_year        0
veh_no          0
make            0
dtype: int64

Eliminar los nulos.

In [ ]:
vh_fn.dropna(inplace=True)
vh_fn.isnull().sum()

accident_id     0
county_name     0
day             0
month           0
year            0
hour            0
minute          0
latitude        0
longitud        0
sp_jur_lit      0
man_coll_lit    0
lgt_cond_lit    0
weather_lit     0
fatals          0
mod_year        0
veh_no          0
make            0
dtype: int64

# 6. Transformaciones: Aplicación de funciones para la creación de nuevas variables.


## 6.1. Crear columna fecha.

In [ ]:
pr_fn['date'] = pd.to_datetime(pr_fn[['year','month','day','hour','minute']])
pr_fn.head()

,accident_id,age,sex,inj_sev,drinking,alc_res,drugs,drugres1,drugres2,drugres3,air_bag,veh_no,mod_year,make,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals,date
0,2016040001,33.0,Male,Suspected Serious Injury (A),1.0,261.0,0.0,Not Reported,Test Not Given,Test Not Given,Deployed-Front,1,2006.0,12.0,PINAL,2.0,1.0,2016.0,13.0,54.0,33.051117,-111.645914,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0,2016-01-02 13:54:00
1,2016040001,26.0,Male,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Deployed-Front,1,2006.0,12.0,PINAL,2.0,1.0,2016.0,13.0,54.0,33.051117,-111.645914,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0,2016-01-02 13:54:00
2,2016040001,36.0,Female,Fatal Injury (K),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2006.0,12.0,PINAL,2.0,1.0,2016.0,13.0,54.0,33.051117,-111.645914,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0,2016-01-02 13:54:00
3,2016040001,10.0,Female,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2006.0,12.0,PINAL,2.0,1.0,2016.0,13.0,54.0,33.051117,-111.645914,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0,2016-01-02 13:54:00
4,2016040001,23.0,Male,Suspected Minor Injury (B),8.0,0.0,8.0,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2006.0,12.0,PINAL,2.0,1.0,2016.0,13.0,54.0,33.051117,-111.645914,Indian Reservation,Not a Collision with Motor Vehicle In-Transport,Daylight,Clear,1.0,2016-01-02 13:54:00


## 6.2. Crear columna que indique si la persona falleció a causa del accidente.

In [ ]:
pr_fn['dead'] = pr_fn['inj_sev'].apply(lambda x: 1 if x=='Fatal Injury (K)' else 0)

## 6.3. Crear columna que indique si la persona estaba drogada al momento del accidente

In [ ]:
si_droga = ['Narcotic', 'Depressant', 'Stimulant', 'Hallucinogen', 'Cannabinoid', 'Phencyclidine (PCP)', 'Anabolic Steroid', 'Inhalant', 'Other Drug', 'Tested for Drugs, Drugs Found, Type Unknown/Positive']
desconocido_droga = ['Test Not Given', 'Tested for Drugs, Results Unknown', 'Not Reported', 'Unknown If Tested']

pr_fn['drugs'] = pr_fn.apply(lambda x: 'Yes' if ((x['drugres1'] in si_droga) or (x['drugres2'] in si_droga) or (x['drugres3'] in si_droga))
                                else 'Unknown' if ((x['drugres1'] in desconocido_droga) or (x['drugres2'] in desconocido_droga) or (x['drugres3'] in desconocido_droga))
                                else 'No', axis=1)

# 7. Análisis descriptivo sobre las bases individuales y su relación

1. ¿Cuál es la cantidad de accidentes de tránsito con muertos por condado?
2. Muestre la cantidad de accidentes de tránsito por año-mes y la cantidad de estos que registran muertos.
3. ¿Cuántos accidentes de tránsito se registran por hora del día, en promedio? ¿Cuántos de estos registran muertos?
4. Indique si hay alguna relación entre los accidentes de tránsito y el estado del clima.
5. ¿Los vehículos más antiguos son los que causan más accidentes? ¿O es el caso contrario?
6. ¿Qué causa más muertes en accidentes de tránsito? ¿Conductores bajo efectos de alcohol o bajo efectos de drogas?
7. ¿Cuántas personas de cada sexo mueren en accidentes de tránsito por año? ¿Cuál es el promedio de edad?
8. ¿Total de accidentes y muertes por marca de vehiculo?
9. ¿La activación del Airbag disminuye la cantidad de muertes en promedio? ¿Que tipo de activación es  mejor? 

## 7.1. ¿Cuál es la cantidad de accidentes de tránsito por condado? ¿Cuántas víctimas fatales hubieron?

In [ ]:
base1 = acc.groupby(['county_name']).agg({'accident_id':'count', 'fatals':'sum'}).reset_index().rename(columns={'accident_id':'accidents'}).sort_values('accidents', ascending=False)
base1

,county_name,accidents,fatals
7,MARICOPA,1867,1987
10,PIMA,465,496
11,PINAL,251,281
2,COCONINO,218,251
8,MOHAVE,196,232
13,YAVAPAI,181,210
0,APACHE,157,187
9,NAVAJO,149,176
14,YUMA,103,115
3,GILA,97,111


In [ ]:
import plotly.express as px

fig = px.bar(base1, x='county_name', y=['accidents', 'fatals'], title='Comparación de Accidentes y Muertes por Condado', barmode='group', text_auto=True)
fig.update_layout(title=dict(text='Accidents and fatals by county', x=0.5))
fig.show()

Se ordenan los condados de mayor a menor número de accidentes, se observa que Maricopa es el condado con más accidentes y muertos, además, se evidencia que hubo más accidentes que fallecidos en cada uno de los condados.

In [ ]:
token_map = "pk.eyJ1IjoiYWxleGJlZCIsImEiOiJjbGgxNHgxdHUwZWJrM21tdzYxNmIzenZnIn0.dDeB0SSguteCnzctb8LCxw"
px.set_mapbox_access_token(token_map)
fig = px.scatter_mapbox(acc, lat='latitude', lon='longitud', color='county_name', mapbox_style='streets',
                        color_continuous_scale=px.colors.cyclical.IceFire, size_max=30, zoom=5)
fig.update_layout(title=dict(text='Accidents by county', x=0.5))
fig.show()

## 7.2. Muestre la cantidad de accidentes de tránsito por año-mes y la cantidad de estos que registran muertos

**Serie de tiempo de accidentes y muertes**

In [ ]:
base2 = acc.groupby(['year', 'month']).agg({'accident_id':'count', 'fatals':'sum'}).reset_index().rename(columns={'accident_id':'accidents'})
year_month = base2.apply(lambda x: str(x['year']) + '-' + str(x['month']), axis =1)
base2 = base2.set_index(year_month).drop(['year', 'month'], axis=1)
base2

,accidents,fatals
2012-1,64,65
2012-2,55,60
2012-3,73,85
2012-4,60,68
2012-5,64,76
2012-6,62,71
2012-7,58,70
2012-8,65,71
2012-9,62,72
2012-10,63,65


In [ ]:
fig = px.line(base2)
fig.update_layout(
    xaxis=dict(title='Month - Year'),
    title=dict(text='Accidents and fatals by month-year', x=0.5)
)
fig.show()

 Se realiza un gráfico de líneas con los accidentes de tránsito y la cantidad de muertes, donde no se puede observar un patrón o ciclo considerable en la línea de tiempo, se nota un leve incremento en los accidentes y muertes del 2015 al 2016, respecto al histórico.

## 7.3. ¿Cuántos accidentes de tránsito se registran por hora del día, en promedio? ¿Cuántos de estos registran muertos?

Se realiza un diagrama de pastel separando los accidentes y las muertes por los momentos del día, donde se toma el día 
desde las 4 horas hasta las 11 horas, la tarde desde las 12 horas hasta las 19 horas y la noche desde las 20 horas hasta las 3 del otro día.

In [ ]:
acc['day_part'] = acc['hour'].apply(lambda x: 'Morning' if x<=11 and x>=4 else 'Afternoon' if x>=12 and x<=19 else 'Night')

In [ ]:
base3 = acc.groupby(['day_part']).agg({'accident_id':'count', 'fatals':'sum'}).reset_index().rename(columns={'accident_id':'accidents'})
base3

,day_part,accidents,fatals
0,Afternoon,1578,1735
1,Morning,999,1103
2,Night,1321,1458


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=base3['day_part'], values=base3['accidents'], name="accidents"),
              1, 1)
fig.add_trace(go.Pie(labels=base3['day_part'], values=base3['fatals'], name="fatals"),
              1, 2)

fig.update_traces(hole=.4, hoverinfo="label+percent+name", textinfo='value')

fig.update_layout(
    annotations=[dict(text='Accidents', x=0.185, y=0.5, font_size=20, showarrow=False),
                 dict(text='Fatals', x=0.80, y=0.5, font_size=20, showarrow=False)],
    title=dict(text='Accidents and fatals by day-part', x=0.5)
                  )
fig.show()

Las horas oras la tarde es el momento donde más frecuentan accidentes y muertes por estos, seguido de la noche y finalmente en la mañana.

## 7.4. Indique si hay alguna relación entre los accidentes de tránsito y el estado del clima.

In [ ]:
acc['weather_lit'].unique()

array(['Clear', 'Cloudy', 'Rain', 'Severe Crosswinds', 'Not Reported',
       'Blowing Sand', 'Snow', 'Sleet or Hail'], dtype=object)

In [ ]:
base4 = acc.groupby(['weather_lit']).agg({'accident_id':'count', 'fatals':'sum'}).reset_index().rename(columns={'accident_id':'accidents'})
base4

,weather_lit,accidents,fatals
0,Blowing Sand,5,5
1,Clear,2953,3262
2,Cloudy,366,404
3,Not Reported,469,502
4,Rain,88,103
5,Severe Crosswinds,6,8
6,Sleet or Hail,4,5
7,Snow,7,7


In [ ]:
fig = px.bar(base4.sort_values('accidents', ascending=False), x='weather_lit', y=['accidents', 'fatals'], barmode='group', text_auto=True)
fig.update_layout(
    title=dict(text='Accidents and fatals by weather', x=0.5)
                  )
fig.show()

Se realiza un gráfico de barras para mostrar el número de accidentes y muertes dependiendo del clima en que suceden los hechos y se observa que la mayoría de accidentes ocurrieron cuando el clima estaba despejado, por lo que se concluye que no hay relación.

## 7.5. ¿Los vehículos más antiguos son los que causan más accidentes? ¿O es el caso contrario?

In [ ]:
base5 = vh_fn.groupby(['mod_year']).agg({'accident_id':'count', 'fatals':'sum'}).reset_index().rename(columns={'accident_id':'accidents'})
base5 = base5.set_index('mod_year')
base5

,accidents,fatals
mod_year,,
1932.0,1,1.0
1956.0,1,1.0
1957.0,1,1.0
1963.0,2,2.0
1965.0,6,6.0
1966.0,1,1.0
1967.0,2,3.0
1968.0,2,2.0
1969.0,1,2.0


In [ ]:
fig = px.line(base5)
fig.update_layout(
    xaxis=dict(title='Model year'),
    yaxis=dict(title='Value'),
    title=dict(text='Accidents and fatals by model year', x=0.5)
)
fig.show()

Para observar la relación que tienen el modelo de los vehículos con el número de accidentes se realiza un gráfico de líneas, donde la línea del tiempo es el modelo del carro, donde se puede ver un aumento considerable en el año 2005 con respecto a los demás años. Los vehículos de modelos entre el 2000 y el 2012, con un notable pico en el año 2005, son los que más se vieron involucrados en accidentes entre los años 2012 y 2016. Lo anterior puede ser debido a que se presenta un mayor número de vehículos de estos modelos circulando en el estado de Arizona.

## 7.6. ¿Qué causa más accidentes de tránsito fatales? ¿Conductores bajo efectos de alcohol o bajo efectos de drogas?

In [ ]:
drugs = pr_fn[pr_fn['drugs']=='Yes'].groupby(['accident_id']).count().shape[0]

In [ ]:
alc = pr_fn[pr_fn['alc_res']>0].groupby(['accident_id']).count().shape[0]

In [ ]:
base6 = pd.DataFrame({'category':['drugs', 'alcohol'], 'accidents':[drugs, alc]})
base6

,category,accidents
0,drugs,950
1,alcohol,2110


Se concluye que los conductores bajo efectos de alcohol causan más accidentes de tránsito que los conductores bajo efectos de drogas.

In [ ]:
fig = px.pie(base6, values='accidents', names='category')
fig.update_layout(width=800, height=400, title=dict(text='Fatal accidents involving drugged or alcoholic people', x=0.5))
fig.show()

## 7.7. ¿Cuántas personas de cada sexo mueren en accidentes de tránsito por año? ¿Cuál es el promedio de edad?

In [ ]:
pr_fn[pr_fn['sex']=='Not Reported']

,accident_id,age,sex,inj_sev,drinking,alc_res,drugs,drugres1,drugres2,drugres3,air_bag,veh_no,mod_year,make,county_name,day,month,year,hour,minute,latitude,longitud,sp_jur_lit,man_coll_lit,lgt_cond_lit,weather_lit,fatals,date,dead
1916,2016040725,2.0,Not Reported,No Apparent Injury (O),8.0,0.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Deployed,2,2008.0,55.0,MARICOPA,12.0,11.0,2016.0,19.0,53.0,33.407317,-111.978281,No Special Jurisdiction,Angle,Dark - Lighted,Clear,1.0,2016-11-12 19:53:00,0
1917,2016040725,1.0,Not Reported,No Apparent Injury (O),8.0,0.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Deployed,2,2008.0,55.0,MARICOPA,12.0,11.0,2016.0,19.0,53.0,33.407317,-111.978281,No Special Jurisdiction,Angle,Dark - Lighted,Clear,1.0,2016-11-12 19:53:00,0
1964,2016040740,38.0,Not Reported,Possible Injury (C),8.0,0.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Applicable,2,2011.0,98.0,MARICOPA,25.0,11.0,2016.0,2.0,30.0,33.461547,-112.411175,No Special Jurisdiction,Front-to-Front,Dark - Lighted,Clear,1.0,2016-11-25 02:30:00,0
2276,2016040863,19.0,Not Reported,"Injured, Severity Unknown",9.0,0.0,Unknown,Not Reported,Test Not Given,Test Not Given,Deployment Unknown,1,2005.0,99.0,COCHISE,16.0,12.0,2016.0,20.0,0.0,31.692464,-110.363769,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Not Reported,Not Reported,1.0,2016-12-16 20:00:00,0
5359,2014040356,1.0,Not Reported,No Apparent Injury (O),8.0,96.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,1998.0,20.0,MARICOPA,12.0,7.0,2014.0,20.0,10.0,33.415444,-111.784233,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dark - Lighted,Clear,1.0,2014-07-12 20:10:00,0
8381,2013040801,30.0,Not Reported,Suspected Serious Injury (A),8.0,96.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Deployment Unknown,1,1999.0,39.0,MARICOPA,13.0,10.0,2013.0,20.0,22.0,33.462142,-112.249678,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dark - Unknown Lighting,Not Reported,1.0,2013-10-13 20:22:00,0
8667,2012040104,5.0,Not Reported,No Apparent Injury (O),8.0,96.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2005.0,20.0,MARICOPA,11.0,2.0,2012.0,20.0,49.0,33.392283,-112.057400,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dark - Lighted,Clear,1.0,2012-02-11 20:49:00,0
8668,2012040104,4.0,Not Reported,No Apparent Injury (O),8.0,96.0,Unknown,Test Not Given,Test Not Given,Test Not Given,Not Applicable,1,2005.0,20.0,MARICOPA,11.0,2.0,2012.0,20.0,49.0,33.392283,-112.057400,No Special Jurisdiction,Not a Collision with Motor Vehicle In-Transport,Dark - Lighted,Clear,1.0,2012-02-11 20:49:00,0


In [ ]:
base7 = pr_fn[pr_fn['dead']==1].groupby(['year','sex']).agg({'dead':'sum', 'age':'mean'})
base7 = base7.reset_index().rename(columns={'dead':'fatals'})
base7

,year,sex,fatals,age
0,2012.0,Female,188,40.611702
1,2012.0,Male,481,40.548857
2,2013.0,Female,201,45.298507
3,2013.0,Male,455,42.909890
4,2014.0,Female,182,46.741758
5,2014.0,Male,405,41.834568
6,2015.0,Female,214,43.747664
7,2015.0,Male,489,43.621677
8,2016.0,Female,206,43.718447
9,2016.0,Male,523,43.608031


In [ ]:
fig = px.line(base7, x='year', y='fatals', color='sex', markers=True)
rango_y = [0, 700]
fig.update_layout(
    title=dict(text='Deads by sex and year', x=0.5),
    yaxis=dict(range=rango_y),
    width=800
)
fig.show()

fig = px.line(base7, x='year', y='age', color='sex', markers=True)
fig.update_layout(
    title=dict(text='Mean age of deads by year', x=0.5),
    width=800
)
fig.show()

 Para identificar las muertes por sexo y año se realiza un gráfico de líneas y se observa que los hombres mueren más en accidentes de tránsito que las mujeres en todos los años. Se observa que las mujeres que fallecieron en accidentes de tránsito tienen un mayor promedio de edad que los hombres, por otra parte, el rango del promedio de edad de los hombres fallecidos es de 41 a 43 años y de las mujeres es de 41 a 47 años.

## 7.8. ¿Cuál es la marca de vehículos que provoca más accidentes de tránsito?

In [ ]:
vh_mk = vh_mk.rename(columns={'Codes': 'make'})
base8 = vh.merge(vh_mk, how='left', on='make')
base8 = base8.rename(columns={'Attributes':'marca_name'}).fillna('Unknown')
base8.head()

,accident_id,mod_year,veh_no,make,marca_name
0,2016040001,2006.0,1,12,Ford
1,2016040002,2005.0,1,Unknown,Unknown
2,2016040003,2015.0,1,37,Honda
3,2016040004,2001.0,1,24,Saturn
4,2016040004,2006.0,2,55,Hyundai


In [ ]:
# crear base
df0 = base8.groupby(['marca_name'])[['accident_id']].count().sort_values('accident_id', ascending = False).rename(columns={'accident_id':'accidents'})
df0['ratio'] = df0.apply(lambda x: x.cumsum()/df0['accidents'].sum())
# definir figura
fig = go.Figure([go.Bar(x=df0.index, y=df0['accidents'], yaxis='y1', name='accidentes de transito'),
                 go.Scatter(x=df0.index, y=df0['ratio'], yaxis='y2', name='accidentes de transito', hovertemplate='%{y:.1%}', marker={'color': '#000000'})])

# agregar detalles
fig.update_layout(template='plotly_white', showlegend=False, hovermode='x', bargap=.3,
                  title={'text': '<b>Total de vehiculos involucrados en accidentes según la marca<b>', 'x': .5}, 
                  yaxis={'title': 'accidentes'},
                  yaxis2={'rangemode': "tozero", 'overlaying': 'y', 'position': 1, 'side': 'right', 'title': 'ratio', 'tickvals': np.arange(0, 1.1, .2), 'tickmode': 'array', 'ticktext': [str(i) + '%' for i in range(0, 101, 20)]})

fig.show()

Podemos observar que más de un 80% de los vehículos involucrados en accidentes se encuentran distribuidos únicamente en el 19%(16) de las marcas de vehículos. Otro dato a destacar es que si tomamos únicamente el 10%(8) de las marcas, estas se han visto involucradas en más del 64% de los accidentes.

## 7.9. ¿La activación del Airbag disminuye la cantidad de muertes en promedio? ¿Que tipo de activación es  mejor? 

In [ ]:
base9 = pr_fn.groupby(['air_bag']).agg({'dead':'sum'}).reset_index().rename(columns={ 'dead':'fatals'})
base9

,air_bag,fatals
0,Deployed-Combination,73
1,Deployed-Curtain (roof),17
2,Deployed-Front,330
3,"Deployed-Other (knee, air belt, etc.)",3
4,"Deployed-Side (door, seatback)",9
5,Deployment Unknown,381
6,Deployment-Unknown Location,170
7,Not Applicable,1324
8,Not Deployed,1022
9,Not Reported,15


In [ ]:
fig = px.pie(base9, values='fatals', names='air_bag')
fig.update_layout(title=dict(text='Fatals by type of airbag activation', x=0.5))
fig.show()

Evidentemente, cuando no se desplegó el airbag, se tuvieron muchas más muertes que con cualquier tipo de activación, por tanto, se concluye que el airbag si disminuye el número de muertes. Por otro lado, los mejores tipos de activación del airbag son otros tipos de activación, que incluyen Knee, Air belt y otros; Activación lateral y Activación de cortina.

# 8. En este punto, pueden complementar su análisis con alguna información adicional? ¿Qué más sería interesante ver en este análisis descriptivo? Buscar información y agregarla

Sería interesante el total de vehículos circulando por marca en los años entre 2012 y 2016 en el estado de Arizona. También sería interesante obtener información del número de autos por año del modelo circulando entre los años 2012 y 2016 en el Estado de arizona.

